In [1]:
import tensorflow as tf
from tensorflow.keras import preprocessing

import numpy as np

In [2]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets =[[1], [0], [1], [1], [0], [1]]

In [3]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
input_sequences = np.array(sequences)
labels = np.array(targets)

word_index = tokenizer.word_index

In [4]:
from tensorflow.keras import backend
from tensorflow.keras import layers

In [5]:
batch_size = 2
num_epochs = 100

In [6]:
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

# Keras Sequential

In [7]:
model = tf.keras.Sequential([
layers.Embedding(vocab_size, emb_size, input_length = 4),
layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
layers.Dense(hidden_dimension, activation='relu'),
layers.Dense(output_dimension, activation='sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 128)            2688      
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6878 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 667us/step - loss: 0.6670 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 667us/step - loss: 0.6489 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 667us/step - loss: 0.6286 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 333us/step - loss: 0.6038 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 333us/step - loss: 0.5785 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 667us/step - loss: 0.5482 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 667us/step - loss: 0.5111 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 667us/step - loss: 0.4701 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 667us/step - loss: 0.4292 - accuracy: 1.0000
Epoch 11/10

# Keras Functional API

In [10]:
inputs = layers.Input(shape = (4, ))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)

In [11]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 4, 128)            2688      
_________________________________________________________________
tf_op_layer_Mean (TensorFlow [(None, 128)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 667us/step - loss: 0.6944 - accuracy: 0.1667
Epoch 2/100
3/3 [==============================] - 0s 667us/step - loss: 0.6751 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 667us/step - loss: 0.6579 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 667us/step - loss: 0.6407 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 334us/step - loss: 0.6238 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 334us/step - loss: 0.6016 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 333us/step - loss: 0.5782 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 667us/step - loss: 0.5478 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 333us/step - loss: 0.5162 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 333us/step - loss: 0.4800 - accuracy: 1.0000
Epoch 11/

# Keras Custom Model

In [15]:
class CustomModel(tf.keras.Model):

    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)
        
        return x

In [16]:
model = CustomModel(vocab_size = vocab_size,
            embed_dimension=emb_size,
            hidden_dimension=hidden_dimension,
            output_dimension=output_dimension)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 667us/step - loss: 0.6907 - accuracy: 0.6667
Epoch 2/100
3/3 [==============================] - 0s 667us/step - loss: 0.6718 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 667us/step - loss: 0.6554 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 667us/step - loss: 0.6366 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 667us/step - loss: 0.6206 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 667us/step - loss: 0.5968 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 334us/step - loss: 0.5712 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 334us/step - loss: 0.5402 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 667us/step - loss: 0.5034 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 667us/step - loss: 0.4678 - accuracy: 1.0000
Epoch 11/

# Keras Custom Layer

In [17]:
class CustomLayer(layers.Layer):

    def __init__(self, hidden_dimension, output_dimension, **kwargs):
        self.hidden_dimension = hidden_dimension
        self.output_dimension = output_dimension
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer2 = layers.Dense(self.output_dimension)

    def call(self, inputs):
        hidden_output = self.dense_layer1(inputs)
        return self.dense_layer2(hidden_output)

    # Optional
    def get_config(self):
        base_config = super(CustomLayer, self).get_config()
        base_config['hidden_dim'] = self.hidden_dimension
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [18]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, emb_size, input_length = 4),
    layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
    CustomLayer(hidden_dimension, output_dimension),
    layers.Activation('sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6915 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 333us/step - loss: 0.6726 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 333us/step - loss: 0.6564 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 334us/step - loss: 0.6387 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 333us/step - loss: 0.6205 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 667us/step - loss: 0.5990 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 667us/step - loss: 0.5751 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 333us/step - loss: 0.5450 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 334us/step - loss: 0.5115 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 667us/step - loss: 0.4736 - accuracy: 1.0000
Epoch 11/10